# Read XML data and store in Excel 

## Set up panda

In [1]:
import pandas as pd
import numpy as np

## Import the NA 1904 XML data

First set up environment to access files on the OS.
XML source files are found on github https://github.com/Clear-Bible/macula-greek

In [25]:
import sys
import os
# MyCurrentDir = os.path.abspath(os.path.join('..'))
MyCurrentDir = "/Users/TF/Downloads/input_xml/"
InputFile = MyCurrentDir+"/sentence.xml"
OutputFile = MyCurrentDir+"/sentence.xlsx"  #excel file

Load the package to handle XML files

In [3]:
!pip install lxml

In [3]:
#import xml.etree.ElementTree as ET
#or (the faster version, need to check compatibility)
import xml.etree.cElementTree as ET

In [26]:
# send xml document to parsing process
tree = ET.parse(InputFile)
root = tree.getroot()

## Add the parent info to each node of the XML tree

In [27]:
# idea from https://stackoverflow.com/questions/2170610/access-elementtree-node-parent-node

def addParentInfo(et):
    for child in et:
        child.attrib['parent'] = et
        addParentInfo(child)

def getParent(et):
    if 'parent' in et.attrib:
        return et.attrib['parent']
    else:
        return None
    
# Now add all the parent info to the nodes in the xtree [important!]

addParentInfo(tree.getroot())

Leaf means the node containing the Greek word as data (i.e. the leaves on the tree).
Parent1 is leaf's parent, Parent2 is Parent1's parent, etc.

## Harvesting xml and storing in MySQL

In [28]:
import time
start_time = time.time()
# walk over all the leaves and harvest the data
# MaxCollectItems (= number of leaves to harvest) is just there for development and debug 
MaxCollectItems = 1000
CollectedItems= 0
# create empty DataFrame
full_df=pd.DataFrame({})
df=pd.DataFrame({})
for elem in tree.iter():
    if (CollectedItems == MaxCollectItems):  break
    if not list(elem):
       if (CollectedItems%25==0): print ("# of leaves:",CollectedItems)
       CollectedItems +=1
       LeafNodeId = elem.attrib.get('nodeId')
       if elem.tag != 'milestone': #remove blank lines
           df=pd.DataFrame({#'LeafNodeId':        [LeafNodeId],
                            #'LeafNodeName':      [elem.tag],                             '
                            #'LeafText':          [elem.text], 
                            'LeafRef':            [elem.attrib.get('ref')],
                            'LeafAfter':          [elem.attrib.get('after')],
                            'LeafClass':          [elem.attrib.get('class')],
                            'LeafNodeId':         [elem.attrib.get('xml:id')],
                            'LeafLemma':          [elem.attrib.get('lemma')],
                            'LeafNormalized':     [elem.attrib.get('normalized')],
                            'LeafStrong':         [elem.attrib.get('strong')],
                            'LeafNumber':         [elem.attrib.get('number')],
                            'LeafGender':         [elem.attrib.get('gender')],
                            'LeafCase':           [elem.attrib.get('case')],
                            'LeafGloss':          [elem.attrib.get('gloss')],  
                            'LeafDomain':         [elem.attrib.get('domain')],
                            'LeafLN':             [elem.attrib.get('ln')],
                            'LeafMorphId':        [elem.attrib.get('morph')],
                            'LeafUnicode':        [elem.attrib.get('unicode')],
                            'LeafJunction':       [elem.attrib.get('junction')],
                            'LeafPerson':         [elem.attrib.get('person')],
                            'LeafTense':          [elem.attrib.get('tense')],
                            'LeafVoice':          [elem.attrib.get('voice')],
                            'LeafMood':           [elem.attrib.get('mood')],
                            'LeafFrame':          [elem.attrib.get('frame')],
                            'LeafArticular':      [elem.attrib.get('articular')], 
                            'LeafDiscontinuos':   [elem.attrib.get('discontinous')],
                            'LeafReferent':       [elem.attrib.get('referent')], 
                            'LeafClauseType':     [elem.attrib.get('clauseType')],
                            'LeafSubjref':        [elem.attrib.get('subjref')],
                            'LeafRule':           [elem.attrib.get('rule')],
                            'LeafnodeId':         [elem.attrib.get('nodeId')],
                            'LeafDegree':         [elem.attrib.get('degree')]
                             },
                            index={LeafNodeId})
           # folling code will trace down parents upto the tree and store found attributes
           parentnode=getParent(elem)
           index=1
           ##parent_df=pd.DataFrame({'LeafNodeId': [LeafNodeId]})
           while (parentnode):
               df=pd.concat([df,pd.DataFrame({
                            'Parent{}Class'.format(index):           parentnode.attrib.get('class'),
                            'Parent{}Rule'.format(index):            parentnode.attrib.get('rule'),
                            'Parent{}Role'.format(index):            parentnode.attrib.get('role'),
                            'Parent{}Type'.format(index):            parentnode.attrib.get('type'),
                            'Parent{}ApposCont'.format(index):       parentnode.attrib.get('appositioncontainer'),
                            },
                            index={LeafNodeId})],
                            axis=1)
               #print("\n------\n", df2)
               currentnode=parentnode
               parentnode=getParent(currentnode)      
               index+=1
           full_df=pd.concat([full_df,df])

#print(full_df)
print ("Finished - Total # of leaves:",CollectedItems)
print("--- %s seconds ---" % (time.time() - start_time))


# of leaves: 0
# of leaves: 25
# of leaves: 50
# of leaves: 75
Finished - Total # of leaves: 96
--- 1.472165822982788 seconds ---


## Export to excel sheet

note: export to excel https://datatofish.com/export-dataframe-to-excel/

if package was not installed, ad it with: pip install openpyxl

In [29]:
full_df.to_excel(r"{}".format(OutputFile), index=False)